In [2]:
import pandas as pd
import itertools as it
import gurobipy as gp
from gurobipy import GRB

In [33]:
data1 = [[100, 250, 95, 160],
         [150, 143, 195, 99],
         [135, 80, 242, 55],
         [83, 225, 111, 96],
         [120, 210, 70, 115],
         [230, 98, 124, 80]]

data2 = [[0.6, 0.20, 0.10, 0.10],
         [0.15, 0.55, 0.25, 0.05],
         [0.15, 0.20, 0.54, 0.11],
         [0.08, 0.12, 0.27, 0.53]]

data3 = [[0, 20, 30, 50],
         [20, 0, 15, 35],
         [30, 15, 0, 25],
         [50, 35, 25, 0]]

data4 = [[50, 70],
         [70, 100],
         [120, 150]]


rentDays = [1, 2, 3]
days = [0, 1, 2, 3, 4, 5]
sat = days[-1]
daysName = dict(zip(days, ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']))
depots = ['Glasgow', 'Manchester', 'Birmingham', 'Plymouth']
repDepots = ['Manchester', 'Birmingham']
nReDepots = ['Glasgow', 'Plymouth']


D = pd.DataFrame(data1, columns=depots, index=days)
P = pd.DataFrame(data2, columns=depots, index=depots)
C = pd.DataFrame(data3, columns=depots, index=depots)
Q = dict(zip(rentDays, [0.55, 0.2, 0.25]))
R  = dict(zip(repDepots, [12, 20]))
RC = pd.DataFrame(data4, columns=['same', 'another'], index=rentDays)
RCSat = dict(zip(['same', 'another'], [30, 50]))
CS = dict(zip(rentDays, [20, 25, 30])) 

In [130]:
model = gp.Model('CarRental1')

# add vars
n  = model.addVar(name='n') # total number of cars

nu = model.addVars(depots, days, name='nu') # undamaged cars at i at day t
nd = model.addVars(depots, days, name='nd') # damaged cars at i at day t

tr = model.addVars(depots, days, name='tr') # rented out cars from i at day t

eu = model.addVars(depots, days, name='eu') # undamaged cars left at i at t
ed = model.addVars(depots, days, name='ed') # damaged cars left at i at t

tu = model.addVars(((i,j,t) for i in depots for j in depots for t in days if i!=j), name='tu') # transfered damaged cars from i to j at t
td = model.addVars(((i,j,t) for i in depots for j in depots for t in days if i!=j), name='td') # transfered damaged cars from i to j at t

rp = model.addVars(depots, days, name='rp') # to be repaired at i at t

model.update()
# set bounds for repair capacity and demand
for i in repDepots:
    for t in days:
        rp[i,t].ub = R[i]

#for i in nReDepots:
#    for t in days:
#        rp[i,t].ub = 0
        
for i in depots:
    for t in days:
        tr[i,t].ub = D[i][t]

# add constraints

    # number of UNDAMAGED cars INTO NON-REPAIR depot i on day t
model.addConstrs((gp.quicksum(0.9*P[i][j]*Q[k]*tr[j,days[t-k]] 
                              for j in depots for k in rentDays) 
                  + gp.quicksum(tu[j,i,days[t-1]] 
                                for j in depots if i != j)
                  + eu[i,days[t-1]]
                  == nu[i,t] for i in nReDepots for t in days),
                name='NR_UN_INTO')

    # number of UNDAMAGED cars OUT NON-REPAIR depot i on day t
model.addConstrs((tr[i,t] 
                  + gp.quicksum(tu[i,j,t] for j in depots if i != j)
                  + eu[i,t]
                  == nu[i,t] for i in nReDepots for t in days),
                name='NR_UN_OUT')

    # number of UNDAMAGED cars INTO REPAIR depot i on day t
model.addConstrs((gp.quicksum(0.9*P[i][j]*Q[k]*tr[j,days[t-k]] 
                              for j in depots for k in rentDays) 
                  + gp.quicksum(tu[j,i,days[t-1]] 
                                for j in depots if i != j)
                  + rp[i,days[t-1]]
                  + eu[i,days[t-1]]
                  == nu[i,t] for i in repDepots for t in days),
                name='R_UN_INTO')

    # number of UNDAMAGED cars OUT REPAIR depot i on day t
model.addConstrs((tr[i,t] 
                  + gp.quicksum(tu[i,j,t] for j in depots if i != j)
                  + eu[i,t]
                  == nu[i,t] for i in repDepots for t in days),
                name='R_UN_OUT')

    # number of DAMAGED cars INTO NON-REPAIR depot i on day t
model.addConstrs((gp.quicksum(0.1*P[i][j]*Q[k]*tr[j,days[t-k]] 
                              for j in depots for k in rentDays) 
                  + ed[i,days[t-1]]
                  == nd[i,t] for i in nReDepots for t in days),
                name='NR_DA_INTO')

    # number of DAMAGED cars OUT NON-REPAIR depot i on day t
model.addConstrs((gp.quicksum(td[i,j,t] 
                              for j in repDepots if i != j)
                  + ed[i,t]
                  == nd[i,t] for i in nReDepots for t in days),
                name='NR_DA_OUT')

    # number of DAMAGED cars INTO REPAIR depot i on day t
model.addConstrs((gp.quicksum(0.1*P[i][j]*Q[k]*tr[j,days[t-k]] 
                              for j in depots for k in rentDays)
                  + gp.quicksum(td[j,i,days[t-1]] 
                                for j in depots if i != j)
                  + ed[i,days[t-1]]
                  == nd[i,t] for i in repDepots for t in days),
                name='R_DA_INTO')

    # number of DAMAGED cars OUT REPAIR depot i on day t
model.addConstrs((rp[i,t]
                  + gp.quicksum(td[i,j,t] 
                                for j in nReDepots if i != j)
                  + ed[i,t]
                  == nd[i,t] for i in repDepots for t in days),
                name='R_DA_OUT')

# limiting the total number of cars
model.addConstr((gp.quicksum(0.25*tr[i,0] + 0.45*tr[i,1]
                             + nu[i,2] + nd[i,2] for i in depots)
                == n), name='totalCars')

# objective function
model.setObjective((gp.quicksum(P[i][i]*Q[k]*(RC['same'][k] - CS[k] + 10)*tr[i,t] 
                                for i in depots for t in days for k in rentDays)
                   + gp.quicksum(P[j][i]*Q[k]*(RC['another'][k] - CS[k] + 10)*tr[i,t] 
                                 for i in depots for j in depots for t in days for k in rentDays if i!=j)
                   #+ gp.quicksum(P[i][i]*Q[1]*(RCSat['same'] - CS[1] + 10)*tr[i,sat] 
                   #              for i in depots)
                   #+ gp.quicksum(P[j][i]*Q[1]*(RCSat['another'] - CS[1] + 10)*tr[i,sat] 
                   #              for i in depots for j in depots if j!=i) 
                   #+ gp.quicksum(P[i][i]*Q[k]*(RC['same'][k] - CS[k] + 10)*tr[i,sat] 
                   #              for i in depots for k in rentDays[1:])
                   #+ gp.quicksum(P[j][i]*Q[k]*(RC['another'][k] - CS[k] + 10)*tr[i,sat] 
                   #              for i in depots for j in depots for k in rentDays[1:] if j!=i)
                   - gp.quicksum(C[j][i]*tu[i,j,t] 
                                 for i in depots for j in depots for t in days if j!=i)
                   - gp.quicksum(C[j][i]*td[i,j,t] 
                                 for i in depots for j in depots for t in days if j!=i)
                   - 15*n),
                  GRB.MAXIMIZE)

model.update()
model.write('Car Rental I.lp')
model.Params.DualReductions = 1
model.optimize()

Parameter DualReductions unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 97 rows, 289 columns and 1061 nonzeros
Model fingerprint: 0xe7131839
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [2e+01, 7e+01]
  Bounds range     [1e+01, 3e+02]
  RHS range        [0e+00, 0e+00]
Presolve removed 49 rows and 85 columns
Presolve time: 0.01s
Presolved: 48 rows, 204 columns, 936 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1102412e+05   3.626166e+02   0.000000e+00      0s
      69    1.2116021e+05   0.000000e+00   0.000000e+00      0s

Solved in 69 iterations and 0.02 seconds
Optimal objective  1.211602072e+05


In [131]:
print('Profit of ${:,.2f}'.format(model.ObjVal))
print('Number of owned cars is {:.0f}'.format(n.x))

report_rented = pd.DataFrame([], columns=depots, index=list(daysName.values()))

for i,t in tr.keys():
    report_rented[i][daysName[t]] = round(tr[i,t].x)
report_rented

Profit of $121,160.21
Number of owned cars is 617


,Glasgow,Manchester,Birmingham,Plymouth
Mon,68,98,95,41
Tue,66,95,155,40
Wed,70,80,123,43
Thu,68,114,111,42
Fri,70,102,70,43
Sat,67,95,124,40


In [132]:
report_damag = pd.DataFrame([], columns=depots, index=list(daysName.values()))
for i,t in nd.keys():
    report_damag[i][daysName[t]] = round(nd[i,t].x)
report_damag

,Glasgow,Manchester,Birmingham,Plymouth
Mon,8,12,20,6
Tue,7,12,20,4
Wed,8,13,20,5
Thu,8,12,21,5
Fri,8,12,20,7
Sat,7,12,22,4


In [133]:
report_undamag = pd.DataFrame([], columns=depots, index=list(daysName.values()))
for i,t in nu.keys():
    report_undamag[i][daysName[t]] = round(nu[i,t].x)
report_undamag

,Glasgow,Manchester,Birmingham,Plymouth
Mon,68,98,146,41
Tue,66,95,155,40
Wed,70,100,123,43
Thu,68,114,116,42
Fri,70,102,124,43
Sat,67,95,158,40
